### Importing libraries

In [70]:
import pandas as pd
import numpy as np
import time 

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)  # me muestre todas las columnas
#pd.set_option('display.max_rows', None)  # me muestre todas las filas

### Getting target list 

In [71]:
datatracks = pd.read_csv('../data/clean/api_tracks1_clean.csv', encoding='latin1')

In [72]:
datatracks.head(2)

,track_id,track_name,album_id,artist_id,artist_name
0,4ZY1RPoLfezEqu03BI2axd,Sgt. Pepper's Lonely Hearts Club Band,1x1jpjDbetGqX0IKCUIBNj,3wGWCP3E3tYqj5memYV9Vq,Bloco do Sargento Pimenta
1,0QLyr8xasb6tgGkdPWYw8d,With a Little Help From My Friends,1x1jpjDbetGqX0IKCUIBNj,3wGWCP3E3tYqj5memYV9Vq,Bloco do Sargento Pimenta


In [73]:
track_ids = datatracks['track_id'].unique().tolist()

In [74]:
track_ids[0:2]

['4ZY1RPoLfezEqu03BI2axd', '0QLyr8xasb6tgGkdPWYw8d']

### Spotify Connection 

In [75]:
#%pip install pyarrow
#%pip install joblib
#%pip install tqdm
#%pip install spotipy
#%pip install python-dotenv
#%pip install requests 

In [76]:
import requests as rq
import pyarrow
from joblib import Parallel, delayed
from tqdm import tqdm
import pymongo
from passwords import *

In [77]:
# Obtener el TOKEN

AUTH_URL = "https://accounts.spotify.com/api/token"

creds = {
    "grant_type": "client_credentials",
    "client_id": Client_ID,
    "client_secret": Client_secret
}

response = rq.post(AUTH_URL, 
                         data=creds, 
                         headers={"Content-Type": "application/x-www-form-urlencoded"})


TOKEN = response.json()['access_token']

# Conexión con la API

url = "https://api.spotify.com/"

headers = {"Authorization": f'Bearer {TOKEN}'}

response = rq.get(url,headers=headers)
response

<Response [200]>

### Getting Tracks Information 

In [78]:
track_ids = datatracks['track_id'].unique().tolist()

In [79]:
track_ids[0:2]

['4ZY1RPoLfezEqu03BI2axd', '0QLyr8xasb6tgGkdPWYw8d']

In [80]:
#endpoint
#https://api.spotify.com/v1/audio-features/{id}

In [81]:
#test 
url = 'https://api.spotify.com/v1/audio-features/0QLyr8xasb6tgGkdPWYw8d'

In [82]:
res = rq.get(url, headers = headers)

In [83]:
resjson = res.json()
type(resjson)

dict

In [84]:
resjson.keys()

dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

In [85]:
resjson['danceability']

0.42

In [86]:
# Construct a list of URLs for querying track audio features
track_feature_urls = [f'https://api.spotify.com/v1/audio-features/{track_id}' for track_id in track_ids]

# Create a list of dictionaries with track information and corresponding feature URLs
track_feature_url_list = [{'track_id': track_id, 'url': url} for track_id, url in zip(track_ids, track_feature_urls)]

In [87]:
# List to store the retrieved track data
track_data_list = []

# Function to search and retrieve track information
def search_track_info(track_url):
    time.sleep(0.40)  # Pause between queries to avoid getting banned

    try:
        response = rq.get(track_url['url'], headers=headers)
        response.raise_for_status()  # Check for request errors
        data = response.json()

        # Extract necessary data from the response
        track_data_list.append({
            'track_id': track_url_list['track_id'],
            'danceability': data['danceability'],
            'popularity': data['popularity'],
            'energy': data['energy'],
            'acousticness': data['acousticness'],
            'valence': data['valence'],
            'instrumentalness': data['instrumentalness'],
            'tempo': data['tempo'],
            'loudness': data['loudness']
        })

    except Exception as e:
        print(f"An error occurred: {e}")
        track_data_list.append({
            'track_id': None,
            'danceability': None,
            'popularity': None,
            'energy': None,
            'acousticness': None,
            'valence': None,
            'instrumentalness': None,
            'tempo': None,
            'loudness': None
        })

# Execute the function for each track URL
for track_url in track_url_list:
    search_track_info(track_url)

# Create a DataFrame from the list of dictionaries
data_tracks2 = pd.DataFrame(track_data_list)

An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/4ZY1RPoLfezEqu03BI2axd
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/0QLyr8xasb6tgGkdPWYw8d
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/1YFlTlQHKR62Sh39ctI7jX
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/1aBpFQAum7vESFuNnWeZZI
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/2OePklaMhBp1kCbbTq5UVk
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/1lTPX7LtskNPbQpOTnUm3J
An error occurred: 429 Client Error: Too Many Requests for url: https://api.spotify.com/v1/tracks/3a2WXzUeNOZKWMlYaboODf


KeyboardInterrupt: 

In [ ]:
data_tracks2.head()

In [ ]:
data_tracks.shape

In [ ]:
data_tracks.to_csv('../data/clean/api_tracks1_clean.csv', index=False)